# Transform files with picca format, to files with desisim format

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import healpy as hp
import desitarget.mock.io as mockio
import lya_mock_functions as mock

In [3]:
# open file with picca format
filename='../example_data/delta_picca/nside_4_pix_0.fits'
#filename='../example_data/delta_picca/z1.85/z1.85_N1000_node_015_nside_4_pix_10.fits'
h = fits.open(filename)
h.info()

Filename: ../example_data/delta_picca/nside_4_pix_0.fits
No.    Name         Type      Cards   Dimensions   Format
  0  PRIMARY     PrimaryHDU       9   (76304, 2048)   float32   
  1  IV          ImageHDU        11   (76304, 2048)   float64   
  2  LOGLAM_MAP  ImageHDU        10   (2048,)   float32   
  3  CATALOG     BinTableHDU     23   76304R x 7C   [E, E, E, E, E, E, 10A]   


In [4]:
#print catalog info
h[3].data

FITS_rec([(67.184296, nan, 3.5867739, 0.0, 0.0, 0.0, '0000580781'),
       (66.028549, nan, 3.5837805, 0.0, 0.0, 0.0, '0000580783'),
       (66.57975, nan, 3.5834749, 0.0, 0.0, 0.0, '0000580784'), ...,
       (73.661453, 76.690735, 3.7589865, 0.0, 0.0, 0.0, '0200293732'),
       (89.650948, 84.195755, 3.7711389, 0.0, 0.0, 0.0, '0200293835'),
       (85.358955, 83.602943, 3.775635, 0.0, 0.0, 0.0, '0200293888')], 
      dtype=(numpy.record, [('RA', '>f4'), ('DEC', '>f4'), ('Z', '>f4'), ('PLATE', '>f4'), ('MJD', '>f4'), ('FIBER', '>f4'), ('THING_ID', 'S10')]))

In [5]:
# get information about quasars (TYPE,RA,DEC,Z_COSMO,DZ_RSD)
catalog = h[3].data
# get arraw with redshift in each cell of grid
loglam = h[2].data
# Get deltas (fluctuation around mean density) 
delta = h[0].data
z_qso = catalog['Z']
print('# initial quasars =',len(z_qso))
# keep only quasars with z>2.0
highz=z_qso>2.0
z_qso = z_qso[highz]
catalog = catalog[highz]
print('original shape',delta.shape)
delta = delta[:,highz]
print(np.min(z_qso),'< z_qso <',np.max(z_qso))
print('# high-z quasars =',len(z_qso))
# keep only quasars with good Dec
bad_dec = np.isnan(catalog['DEC']) | (catalog['DEC'] < -90.0) | (catalog['DEC'] > 90.0)
catalog = catalog[np.invert(bad_dec)]
z_qso = z_qso[np.invert(bad_dec)]
delta = delta[:,np.invert(bad_dec)]
Nq=len(z_qso)
print('# good quasars =',len(z_qso))
wave=np.power(10.0,loglam)
z = wave/1215.67-1.0
Nz=len(z)
print('full wavelength shape',delta.shape)
# we will only write pixels with wavelength in DESI spectrograph
in_desi=wave>3550.0
z = z[in_desi]
wave = wave[in_desi]
delta = delta[in_desi]
print('DESI shape',delta.shape)

# initial quasars = 76304
original shape (2048, 76304)
2.00008 < z_qso < 3.79
# high-z quasars = 32306
# good quasars =

/Users/font/Programs/Others/anaconda3/envs/desi/lib/python3.5/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in less
/Users/font/Programs/Others/anaconda3/envs/desi/lib/python3.5/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in greater


 28214
full wavelength shape (2048, 28214)
DESI shape (571, 28214)


In [6]:
# identify HEALPix pixels for our quasars
nside=8
Npix=12*nside*nside
pixels=hp.ang2pix(nside,(catalog['DEC']+90.0)/180.0*np.pi,catalog['RA']/180.0*np.pi)

In [7]:
nonzero_pixels=[]
for pix in range(Npix):
    in_pix=(pixels==pix)
    if len(catalog[in_pix]): 
        nonzero_pixels.append(pix)
print('relevant pixels',nonzero_pixels)

relevant pixels [745, 756, 757, 764]


In [8]:
transmission_base_dir='test_dir_'+str(nside)
for pix in range(Npix):
    # get quasars in HEALPix pixel
    in_pix=(pixels==pix)
    # if none, move on
    if len(catalog[in_pix]) == 0:
        continue
    # select relevant quasars
    qso_in_pix = catalog[in_pix]
    delta_in_pix = delta[:,in_pix]
    print('delta shape',delta_in_pix.shape)
    N_in_pix=len(qso_in_pix)
    print('useful pixel',pix,N_in_pix)

    #Add a couple of headers to the file.
    header = fits.Header()
    header['NSIDE'] = nside
    header['NQSO'] = N_in_pix
    header['PIX'] = int(pix)
    prim_hdu = fits.PrimaryHDU(header=header)

    #meta-data
    ra=qso_in_pix['RA']
    dec=qso_in_pix['DEC']
    zq=qso_in_pix['Z']
    mockid=qso_in_pix['THING_ID']
    
    #Construct a table for the meta-data hdu
    col_ra = fits.Column(name='RA', array=ra, format='E')
    col_dec = fits.Column(name='DEC', array=dec, format='E')
    col_zq = fits.Column(name='Z', array=zq, format='E')
    col_id = fits.Column(name='MOCKID', array=mockid, format='A10')
    meta_hdu = fits.BinTableHDU.from_columns([col_ra, col_dec, col_zq, col_id],name='METADATA')
    #meta_hdu.writeto('test_'+str(pix)+'.fits')
        
    flux = np.empty_like(delta_in_pix)
    for i in range(N_in_pix):
        # Convert density to flux
        tau = mock.get_tau(z,1+delta_in_pix[:,i])
        toflux = np.exp(-tau)
        # only add absorption in the forest 
        no_forest = (z > z_qso[i])
        toflux[no_forest]=1.0
        flux[:,i]=toflux
    
    wave_hdu = fits.ImageHDU(data=wave,name='WAVELENGTH')
    flux_hdu = fits.ImageHDU(data=flux,name='TRANSMISSION')
    hdulist = fits.HDUList([prim_hdu,meta_hdu,wave_hdu,flux_hdu])

    # open file to write
    #filename='test_'+str(pix)+'.fits'
    dirname=mockio.get_healpix_dir(nside, pix, basedir=transmission_base_dir)
    #print('dirname',dirname)
    os.makedirs(dirname, exist_ok=True)
    filename=mockio.findfile('transmission', nside, pix, transmission_base_dir)
    #print('filename',filename)
    #fits = fitsio.FITS(filename,'rw',clobber=True)
    hdulist.writeto(filename)

delta shape (571, 7039)
useful pixel 745 7039
delta shape (571, 7266)
useful pixel 756 7266
delta shape (571, 6993)
useful pixel 757 6993
delta shape (571, 6916)
useful pixel 764 6916


In [9]:
master_file=transmission_base_dir+"/master.fits"
print('master file',master_file)

#Add a couple of headers to the file.
header = fits.Header()
header['NSIDE'] = nside
header['NQSO'] = Nq

#Construct a table for the meta-data hdu
col_ra = fits.Column(name='RA', array=catalog['RA'], format='E')
col_dec = fits.Column(name='DEC', array=catalog['DEC'], format='E')
col_zq = fits.Column(name='Z', array=catalog['Z'], format='E')
col_id = fits.Column(name='MOCKID', array=catalog['THING_ID'], format='A10')
col_pix = fits.Column(name='PIXNUM', array=pixels, format='J')
meta_hdu = fits.BinTableHDU.from_columns([col_ra, col_dec, col_zq, col_id, col_pix],name='METADATA',header=header)
meta_hdu.writeto(master_file)

master file test_dir_8/master.fits
